In [164]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import re


In [165]:
# Tính toán Jaccard similarity
def jaccard_similarity(query, document):
    query_words = set(re.split(r'\s+', query.lower().strip()))
    doc_words = set(re.split(r'\s+', document.lower().strip()))
    intersection = query_words.intersection(doc_words)
    union = query_words.union(doc_words)
    return len(intersection) / len(union)

In [166]:
df = pd.read_csv('questions_answers.csv')

# Tách câu hỏi và câu trả lời
questions = df['question']
answers = df['answer']
print(questions)

0      Nguyên tắc xét tuyển của học viện năm nay ntn?...
1                       Câu hỏi thường gặp về tuyển sinh
2      Chương trình đào tạo tại miền Nam có khác với ...
3      Trường mình có 2 cơ sở, vậy sẽ phân bố học sin...
4      Em đăng ký NV1 là ngành An toàn thông tin, NV2...
                             ...                        
561    Điểm chuẩn tất cả các ngành theo phương thức x...
562         Học viện có bao nhiêu phương thức tuyển sinh
563                                  Hello Hi, chào bot!
564                                            Hello Hi!
565    Diểm chuẩn ngành công nghệ internet vạn vật (i...
Name: question, Length: 566, dtype: object


In [167]:
# Tạo bộ từ điển các từ viết tắt
abbreviation_dict = {
    "cntt": "công nghệ thông tin",
    "attt": "an toàn thông tin",
    "iot": "công nghệ internet vạn vật",
    "fintech": "công nghệ tài chính",
    "cndpt": "công nghệ đa phương tiện",
    "ttdpt": "truyền thông đa phương tiện",
    "qtkd": "quản trị kinh doanh",
    "tmdt": "thương mại điện tử",
    "khmt": "khoa học máy tính",
    "clc": "chất lượng cao",
}

# Hàm để thay thế các từ viết tắt trong câu bằng từ đầy đủ
def replace_abbreviations(text, abbreviation_dict):
    words = text.split()
    new_words = []
    for word in words:
        if word in abbreviation_dict:
            new_words.append(abbreviation_dict[word])
        else:
            new_words.append(word)
    return ' '.join(new_words)

# Ví dụ sử dụng
user_input = "Tôi đang học cntt và muốn tìm hiểu về ngành clc"
processed_input = replace_abbreviations(user_input, abbreviation_dict)

print("Trước khi xử lý:", user_input)
print("Sau khi xử lý:", processed_input)


Trước khi xử lý: Tôi đang học cntt và muốn tìm hiểu về ngành clc
Sau khi xử lý: Tôi đang học công nghệ thông tin và muốn tìm hiểu về ngành chất lượng cao


In [168]:
import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

remove_punctuation("Chi tiết về     Xét tuyển tài năng?@")

'Chi tiết về     Xét tuyển tài năng'

In [169]:
def to_lowercase(text):
    return text.lower()
to_lowercase('Chi tiết về     Xét tuyển tài năng')


'chi tiết về     xét tuyển tài năng'

In [170]:
from pyvi import ViTokenizer
def tokenizerText(text):
    return ViTokenizer.tokenize(text)
tokenizerText('chi tiết về     xét tuyển tài năng ')

'chi_tiết về xét tuyển tài_năng'

In [171]:
tokenizerText("Học máy rất thú vị.")

'Học máy rất thú_vị .'

In [172]:
# Khởi tạo tập hợp để lưu trữ các từ dừng
stopwords = set()

# Đọc từng hàng trong file stopwords.txt
with open('stopwords.txt', 'r', encoding='utf-8') as fp:
    for line in fp:
        word = line.strip() # Loại bỏ khoảng trắng đầu và cuối dòng
        if word:  # Kiểm tra xem dòng không rỗng
            stopwords.add(word)

# In danh sách các từ dừng
print(stopwords)
print(len(stopwords))


{'bởi_vậy', 'biết_mình', 'bỗng_không', 'biết_chừng_nào', 'ngay_thật', 'cho_tới_khi', 'cho_nên', 'biết_bao_nhiêu', 'bị', 'bỗng_nhiên', 'cho_đến', 'là', 'cho_tin', 'biết', 'bỗng_đâu', 'là_thế_nào', 'cho_rằng', 'ngay_cả', 'bỗng', 'ngay_bây_giờ', 'bỗng_dưng', 'bây_giờ', 'bởi', 'bởi_đâu', 'bởi_vì', 'bây_nhiêu', 'biết_mấy', 'cho_về', 'biết_đâu_đấy', 'ngay_khi', 'cho_nhau', 'cho_rồi', 'cho_đến_khi', 'biết_thế', 'dù_cho', 'bởi_ai', 'làm_bằng', 'cho_ăn', 'ngay_lúc_này', 'là_vì', 'biết_bao', 'dù_rằng', 'ngay_tức_khắc', 'bởi_sao', 'biết_chắc', 'cho_biết', 'là_phải', 'biết_việc', 'bởi_thế', 'bởi_tại', 'biết_đâu', 'bởi_nhưng', 'bấy_giờ', 'dù_dì', 'cho_chắc', 'bấy_nhiêu', 'bởi_thế_cho_nên', 'cho_tới', 'làm_cho', 'bởi_chưng', 'cho', 'là_ít', 'biết_trước', 'bao_lâu', 'bỗng_nhưng', 'là_là', 'dù_sao', 'cho_đang', 'ngay_lập_tức', 'biết_đâu_chừng', 'là_cùng', 'ngay_lúc', 'bấy_lâu', 'bấy_lâu_nay', 'cho_được', 'cho_thấy', 'bỗng_thấy', 'làm', 'bỗng_chốc', 'dù_gì', 'ngay_khi_đến', 'cho_đến_nỗi', 'bao_giờ', 'b

In [173]:
def remove_stopwords(line):
    words = []
    for word in line.split():
        if word not in stopwords:
            words.append(word)
    return ' '.join(words)
text = 'Điểm chuẩn ngành cntt theo phương thức thi thpt 2023 là bao nhiêu?'
print(tokenizerText(text))
print(remove_stopwords(tokenizerText(text)))

Điểm_chuẩn ngành cntt theo phương_thức thi thpt 2023 là bao_nhiêu ?
Điểm_chuẩn ngành cntt theo phương_thức thi thpt 2023 ?


In [174]:
def processing_text_for_db(text):
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = tokenizerText(text)
    text = remove_stopwords(text)
    return text

In [175]:
def processing_text_for_query(text):
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_abbreviations(text, abbreviation_dict)
    text = tokenizerText(text)
    text = remove_stopwords(text)
    return text

Demo preprocessing question

In [176]:
q = "Điểm chuẩn ngành cntt clc năm 2023 theo phương thức thi thpt tại cơ sở miền bắc   là bao   nhiêu ?!$"

print(processing_text_for_query(q))



điểm_chuẩn ngành công_nghệ thông_tin chất_lượng cao năm 2023 theo phương_thức thi thpt tại cơ_sở miền bắc


In [177]:
processed_questions = []
for question in questions:  # Sử dụng tqdm để theo dõi tiến trình nếu cần
    # Áp dụng các hàm tiền xử lý
    processed_text = processing_text_for_db(question)
   
    # Lưu kết quả vào list processed_questions
    processed_questions.append(processed_text)

# Thêm cột mới vào DataFrame hoặc ghi đè lên cột 'question' hiện tại
df['processed_question'] = processed_questions

# In ra vài dòng đầu của DataFrame sau khi đã tiền xử lý
print(df.head())

                                            question  \
0  Nguyên tắc xét tuyển của học viện năm nay ntn?...   
1                   Câu hỏi thường gặp về tuyển sinh   
2  Chương trình đào tạo tại miền Nam có khác với ...   
3  Trường mình có 2 cơ sở, vậy sẽ phân bố học sin...   
4  Em đăng ký NV1 là ngành An toàn thông tin, NV2...   

                                              answer  \
0  I. Xét tuyển dựa vào kết quả thi THPT 2024:\n-...   
1                            Bạn đang gặp vấn đề gì?   
2  Chương trình đào tạo của Học viện được áp dụng...   
3  Chỉ tiêu đào tạo của từng cơ sở đã được Học vi...   
4  Không được em nhé! Theo nguyên tắc xét tuyển t...   

                                  processed_question  
0  nguyên_tắc xét tuyển của học_viện năm nay ntn ...  
1                   câu hỏi thường gặp về tuyển_sinh  
2  chương_trình đào_tạo tại miền nam có khác với ...  
3  trường mình có 2 cơ_sở vậy sẽ phân_bố học_sinh...  
4  em đăng_ký nv1 ngành an_toàn thông_tin nv2 ngà..

In [178]:
# import torch
# from transformers import AutoModel, AutoTokenizer

# phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
# tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# # Function to encode each question into vector representation
# def encode_question(question):
#     # Tokenize the question
#     tokens = tokenizer.tokenize(question)
#     # Encode tokens to input_ids
#     input_ids = tokenizer.convert_tokens_to_ids(tokens)
#     # Convert to PyTorch tensor
#     input_ids = torch.tensor([input_ids])
#     # Get features from PhoBERT
#     with torch.no_grad():
#         features = phobert(input_ids)
#     # Return the representation vector of the sentence
#     return features[0].squeeze().tolist()

# df['question_vector'] = questions.apply(encode_question)
# print(df.head())



In [179]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# Function to encode each question into vector representation
def encode_question(question):
   # Tokenize và chuyển đổi câu văn bản thành tensor
    tokens = tokenizer(question, return_tensors='pt', padding=True, truncation=True)

    # Sử dụng model để mã hóa câu văn bản thành vector
    with torch.no_grad():
        output = phobert(**tokens)

    # Lấy vector biểu diễn từ outputs của model
    return output.last_hidden_state.mean(dim=1).numpy()  # Trung bình các vector token

df['question_vector'] = questions.apply(encode_question)
print(df.head())



Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


                                            question  \
0  Nguyên tắc xét tuyển của học viện năm nay ntn?...   
1                   Câu hỏi thường gặp về tuyển sinh   
2  Chương trình đào tạo tại miền Nam có khác với ...   
3  Trường mình có 2 cơ sở, vậy sẽ phân bố học sin...   
4  Em đăng ký NV1 là ngành An toàn thông tin, NV2...   

                                              answer  \
0  I. Xét tuyển dựa vào kết quả thi THPT 2024:\n-...   
1                            Bạn đang gặp vấn đề gì?   
2  Chương trình đào tạo của Học viện được áp dụng...   
3  Chỉ tiêu đào tạo của từng cơ sở đã được Học vi...   
4  Không được em nhé! Theo nguyên tắc xét tuyển t...   

                                  processed_question  \
0  nguyên_tắc xét tuyển của học_viện năm nay ntn ...   
1                   câu hỏi thường gặp về tuyển_sinh   
2  chương_trình đào_tạo tại miền nam có khác với ...   
3  trường mình có 2 cơ_sở vậy sẽ phân_bố học_sinh...   
4  em đăng_ký nv1 ngành an_toàn thông_tin nv2 

In [180]:
fifth_element = df['question_vector'].iloc[324]
print(df['question'].iloc[324])
print(df['processed_question'].iloc[324])
print(df['processed_question'].iloc[313])

print(fifth_element.shape)
print(fifth_element)


Điểm chuẩn ngành thương mại điện tử theo phương thức thi thpt năm 2023 tại cơ sở miền bắc là bao nhiêu?
điểm_chuẩn ngành thương_mại điện_tử theo phương_thức thi thpt năm 2023 tại cơ_sở miền bắc
điểm_chuẩn ngành kỹ_thuật điều_khiển và tự_động hóa theo phương_thức thi thpt năm 2023 tại cơ_sở miền bắc
(1, 768)
[[ 1.27193093e-01  8.82265121e-02 -1.49289161e-01 -6.89974502e-02
  -2.02951819e-01  4.06952798e-02  7.15420544e-02  2.86344998e-02
  -2.01561034e-01  4.24265079e-02 -3.33244130e-02  1.63626552e-01
  -1.26928166e-02 -1.47357374e-01 -8.71807858e-02 -4.88565229e-02
   1.37584088e-02  1.19853482e-01  1.21699087e-01 -4.84706461e-02
  -9.00105387e-02  9.01822895e-02 -2.42922478e-03  3.51708025e-01
  -1.98031783e-01  6.57050696e-04  7.77249262e-02  2.79813766e-01
  -3.79326418e-02 -1.15963332e-01 -5.64399399e-02 -2.04055399e-01
  -2.31248736e-02  1.48644000e-01  4.07536954e-01  4.32585955e-01
  -6.52193576e-02  1.77917350e-02  7.34209418e-02  2.52819419e-01
   4.85257804e-02  2.00461358e-

In [181]:
query = "Điểm chuẩn ngành an toàn thông tin theo phương thức thi thpt 2023 miền bắc?"
processed_query = processing_text(query)
print(processed_query)
query_vector = encode_question(processed_query)
print(query_vector.shape)
print(query_vector)


điểm_chuẩn ngành an_toàn thông_tin theo phương_thức thi thpt 2023 miền bắc
(1, 768)
[[ 2.47866198e-01  8.75657946e-02 -1.72575355e-01 -1.28289014e-01
  -1.36528999e-01  2.49251686e-02  2.10783035e-02  1.29400179e-01
  -1.74124524e-01  8.21061879e-02 -1.68095723e-01  1.89115480e-01
   4.06486169e-02 -1.03111692e-01 -5.82254268e-02  4.59612384e-02
  -1.04514919e-01  2.47891799e-01  2.09378958e-01 -1.13384359e-01
  -2.05431148e-01  1.34707972e-01  8.07819441e-02  4.12992805e-01
  -9.81463119e-02 -1.18047111e-01 -4.29528728e-02  2.61191756e-01
  -1.53154209e-01 -1.34863913e-01 -1.52581677e-01 -2.25703314e-01
  -1.42768115e-01  1.45331517e-01  4.31122392e-01  4.21756595e-01
  -6.69411942e-02 -5.49872033e-02  1.53473541e-01  3.05534843e-02
  -1.26891593e-02  3.01260233e-01  6.55282801e-03 -1.20831057e-01
   2.02900425e-01  1.56172484e-01  1.58965588e-02  8.28259960e-02
   4.37981971e-02  2.17373222e-01  2.27199525e-01  2.87639171e-01
   4.94743623e-02  3.82300884e-01  1.01569042e-01  1.54131

In [182]:
from sklearn.metrics.pairwise import cosine_similarity


# Calculate cosine similarity
cosine_similarities = cosine_similarity(query_vector,fifth_element)

# Print cosine similarities
print("Cosine similarities:")
print(cosine_similarities)




Cosine similarities:
[[0.88356024]]


In [188]:

questions_vector = df['question_vector']
processed_questions = df['processed_question']  
questions = df['question']
def get_response(user_query):
    processed_query = processing_text_for_query(user_query)
    query_vector = encode_question(processed_query)
    # Tính toán độ tương đồng cosine giữa câu truy vấn của người dùng và các câu hỏi trong database
    cosine_similarities = [cosine_similarity(
        query_vector, qv).flatten() for qv in questions_vector]
    jaccard_similarities = [jaccard_similarity(
        processed_query, q) for q in  df['processed_question']]

    # Kết hợp kết quả từ hai độ tương đồng
    alpha = 0.6
    beta = 0.4
    # Chuyển đổi jaccard_similarities thành mảng một chiều
    jaccard_array = np.array(jaccard_similarities).reshape(len(df['processed_question']),1)

    # Kết hợp điểm số từ hai độ tương đồng
    combined_scores = alpha * np.array(cosine_similarities) + beta * jaccard_array

    best_match_index = np.argmax(combined_scores)
    if np.max(combined_scores) > 0.6:
        print(np.max(combined_scores))
        print(processed_query)
        print(processed_questions[best_match_index])
        print(questions[best_match_index])
        return answers[best_match_index]
    else:
        return "Tôi không hiểu câu hỏi của bạn. Vui lòng đặt câu hỏi đầy đủ hơn."
user_input = "điểm chuẩn ngành iot theo phương thức thi thpt 2022  ??"
response = get_response(user_input)
print(response)

# # Chạy chatbot
# user_input = input("You: ")
# if user_input.lower() == "exit":
#     break
# response = get_response(user_input)
# print(f"Bot: {response}")

0.7649182081222534
điểm_chuẩn ngành công_nghệ internet vạn_vật theo phương_thức thi thpt 2022
điểm_chuẩn ngành công_nghệ internet vạn_vật iot theo phương_thức thi thpt năm 2022 tại cơ_sở miền nam
Điểm chuẩn ngành công nghệ internet vạn vật (iot) theo phương thức thi thpt năm 2022 tại cơ sở miền nam là bao nhiêu?
Điểm chuẩn ngành công nghệ internet vạn vật (iot) theo phương thức thi thpt năm 2022 tại cơ sở miền nam là 20.7


In [184]:
jc = jaccard_similarity(processed_query,df['processed_question'].iloc[313] )
print(np.array([[jc]]))
print(np.array([[jc]]).shape)

print(cosine_similarities.shape)
print(np.array(cosine_similarities).shape)


[[0.47368421]]
(1, 1)
(1, 1)
(1, 1)


In [185]:
questions = df['question']
processed_questions = df['processed_question']  
print(questions.iloc[313])
print(processed_questions.iloc[313])

Điểm chuẩn ngành kỹ thuật điều khiển và tự động hóa theo phương thức thi thpt năm 2023 tại cơ sở miền bắc là bao nhiêu?
điểm_chuẩn ngành kỹ_thuật điều_khiển và tự_động hóa theo phương_thức thi thpt năm 2023 tại cơ_sở miền bắc
